In [ ]:
from sbo_steps1to3_modular import execute_multiple_runs, warm_start_nft, auto_resume
from doe import doe

for key in sorted(doe.keys()):
    print(key)

/Users/andrypaez/projects/WISER_Optimization_VG
1/109bonds/TwoLocal1rep_piby3_AerSimulator_0.1
1/109bonds/TwoLocal2rep_bilinear_piby3_fez_0.1
1/109bonds/TwoLocal2rep_color_piby3_fez_0.1
1/109bonds/TwoLocal2rep_color_piby3_marrakersh_0.1
1/109bonds/TwoLocal2rep_piby3_AerSimulator_0.1
1/109bonds/bfcdR2rep_color_piby3_marrakesh_0.1
1/155bonds/TwoLocal2rep_piby3_AerSimulator_0.1
1/31bonds/TwoLocal1repFull_piby3_AerSimulator_0.1
1/31bonds/TwoLocal1rep_piby3_AerSimulator_0.1
1/31bonds/TwoLocal1rep_piby3_AerSimulator_0.15
1/31bonds/TwoLocal1rep_piby3_AerSimulator_0.2
1/31bonds/TwoLocal2repFull_piby3_AerSimulator_0.1
1/31bonds/TwoLocal2rep_piby3_AerSimulator_0.1
1/31bonds/TwoLocal2rep_piby3_AerSimulator_0.15
1/31bonds/TwoLocal2rep_piby3_AerSimulator_0.2
1/31bonds/TwoLocal2rep_piby3_kyiv_0.1
1/31bonds/TwoLocal2rep_piby3_kyiv_0.15
1/31bonds/TwoLocal3repFull_piby3_AerSimulator_0.1
1/31bonds/TwoLocal3repFull_piby3_AerSimulator_0.15
1/31bonds/TwoLocal3repFull_piby3_AerSimulator_0.2
1/31bonds/TwoLoc

In [10]:
doe["1/31bonds/test"]

{'lp_file': '/Users/andrypaez/projects/WISER_Optimization_VG/data/1/31bonds/docplex-bin-avgonly.lp',
 'num_exec': 1,
 'ansatz': 'TwoLocal',
 'theta_initial': 'piby3',
 'optimizer': 'nft',
 'device': 'AerSimulator',
 'max_epoch': 1,
 'shots': 8192,
 'theta_threshold': 0.06,
 'experiment_id': 'test',
 'ansatz_params': {'reps': 1, 'entanglement': 'bilinear'},
 'alpha': 0.1}

In [8]:
from pathlib import Path

ROOT = Path.cwd().parent
print(ROOT)

/Users/andrypaez/projects/WISER_Optimization_VG


#### Step 1-3

In [ ]:
# only for test case in notebook, not best practice
import nest_asyncio
nest_asyncio.apply()

execute_multiple_runs(**doe['1/31bonds/test'], instance='', run_on_serverless=False)

31 15


2025-08-02 02:37:46,722 INFO optimization_wrapper: run...
2025-08-02 02:37:46,722 DEBUG optimization_wrapper: Using X0 [1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755 1.04719755
 1.04719755 1.04719755]
2025-08-02 02:37:46,723 INFO nft: Using parameters: max_epoch 1
2025-08-02 02:37:46,723 INFO nft: Using parameters: random_update True
2025-08-02 02:37:46,723 INFO nft: Using parameters: 

#### Step 4

In [13]:
from sbo_step4 import *

doe = doe_localsearch['unconstrained']
experiments: list[Experiment] = Experiment.read_experiments('1/31bonds/test') 

for xp in experiments:
    if xp.has_step4():
        continue

    lp_file = xp.lp_file.replace('/Users/andrypaez/projects/WISER_Optimization_VG', str(ROOT))
    out_file = Path(lp_file).parent / f'{xp.experiment_id.replace("/", "/exp")}_LS{doe["local_search_doe"]}.pkl'
    
    if out_file.exists():
        continue
    
    with open(out_file, 'bw') as f: # placeholder file
        pkl.dump({}, f)

    # step 1
    # obj_fn, ansatz_, theta_initial_, backend, initial_layout = problem_mapping(lp_file.replace('.lp','-nocplexvars.lp'), xp.ansatz, xp.ansatz_params, xp.theta_initial, xp.device, '')

    model: docplex.mp.model.Model = docplex.mp.model_reader.ModelReader.read(lp_file.replace('.lp','-nocplexvars.lp'))

    obj_fn = model_to_obj(model)
    num_vars = model.number_of_binary_variables


    if 'local_search_maxfevals_per_variable' in doe:
        doe['local_search_maxfevals'] = num_vars * doe.pop('local_search_maxfevals_per_variable')

    # step 2 (not needed)

    # step 3 (not needed)

    # step 4

    iter_best_fx = []
    iter_fx_evals = []
    result_best_x, result_best_val = None, None
    step4_time = 0

    # pathlist = Path(xp.lp_file).parent.glob(f'{xp.experiment_id}_*.pkl')
    for iter in range(len(xp.step3_monitor_iter_best_fx)):
        print(xp.experiment_id, ' - iter ', iter)

        x_best_per_job, fx_best_per_job, feval_iter, list_processed_x_iter, list_processed_fx_iter, proc_time =\
            postprocess_iter(Path(lp_file).parent / f'{xp.experiment_id}_{iter}.pkl', doe, num_vars, obj_fn, xp.refvalue, doe["local_search_doe"])
        
        if len(fx_best_per_job) == 0:
            iter_best_fx.append(None)
            iter_fx_evals.append(None)

            step4_time += proc_time
            continue

        best_iter = np.argmin(fx_best_per_job)
        if result_best_val is None or fx_best_per_job[best_iter] < result_best_val:
            result_best_x, result_best_val = x_best_per_job[best_iter], fx_best_per_job[best_iter]

        iter_best_fx.append(fx_best_per_job)
        iter_fx_evals.append(feval_iter)

        step4_time += proc_time
    
    xp.local_search_doe = doe["local_search_doe"]
    xp.step4_time = step4_time
    xp.step4_num_epochs = None
    xp.step4_fx_evals = iter_fx_evals
    xp.step4_result_best_x = result_best_x
    xp.step4_result_best_fx = result_best_val
    xp.step4_iter_best_fx = iter_best_fx

    with open(out_file, 'bw') as f:
        pkl.dump(dataclasses.asdict(xp), f)

31 15
test/0  - iter  0
test/0  - iter  1
test/0  - iter  2
test/0  - iter  3
test/0  - iter  4
test/0  - iter  5
test/0  - iter  6
test/0  - iter  7
test/0  - iter  8
test/0  - iter  9
test/0  - iter  10
test/0  - iter  11
test/0  - iter  12
test/0  - iter  13
test/0  - iter  14
test/0  - iter  15
test/0  - iter  16
test/0  - iter  17
test/0  - iter  18
test/0  - iter  19
test/0  - iter  20
test/0  - iter  21
test/0  - iter  22
test/0  - iter  23
test/0  - iter  24
test/0  - iter  25
test/0  - iter  26
test/0  - iter  27
test/0  - iter  28
test/0  - iter  29
test/0  - iter  30
test/0  - iter  31
test/0  - iter  32
test/0  - iter  33
test/0  - iter  34
test/0  - iter  35
test/0  - iter  36
test/0  - iter  37
test/0  - iter  38
test/0  - iter  39
test/0  - iter  40
test/0  - iter  41
test/0  - iter  42
test/0  - iter  43
test/0  - iter  44
test/0  - iter  45
test/0  - iter  46
test/0  - iter  47
test/0  - iter  48
test/0  - iter  49
test/0  - iter  50
test/0  - iter  51
test/0  - iter  